In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers --quiet
!pip install sentence_transformers --quiet
!pip install tqdm --quiet
!pip install datasets --quiet
!pip install evaluate --quiet
!pip install peft --quiet

In [3]:
!pip install emoji --quiet

In [4]:
import pandas as pd
import numpy as np
import glob
from sklearn.model_selection import train_test_split
import re
import emoji
import argparse
import os
import evaluate
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader,Dataset
import os

import os

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)

import evaluate
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from tqdm import tqdm

In [5]:
import os
path = r"/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/*.csv" # use your path
all_files = glob.glob(path)
all_files

['/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2011Tornado_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2016Matthew_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2018Wildfires_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2013Floods_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2012Sandy_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2017Hurricane_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2016Blizzard_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2018Michael_Summary.csv',
 '/content/drive/MyDrive/Natural-Hazards-Twitter-Dataset/2019Dorian_Summary.csv']

In [6]:
filenames=[]
for p in all_files:
    filenames.append(p.split("/")[-1])
    
df_dict = dict()
for n,f in zip(filenames,all_files):
    df_dict[n] = pd.read_csv(f)
    if n=='2018Michael_Summary.csv':
      df_dict[n]['num']=df_dict[n]['Unnamed: 0']
      df_dict[n]['label']=df_dict[n]['sentiment']
      df_dict[n].drop(['sentiment','Unnamed: 0'],axis=1,inplace=True)
    df_dict[n]['filename']=[n]*len(df_dict[n])
   

In [7]:
df_dict.keys()

dict_keys(['2011Tornado_Summary.csv', '2016Matthew_Summary.csv', '2018Wildfires_Summary.csv', '2013Floods_Summary.csv', '2012Sandy_Summary.csv', '2017Hurricane_Summary.csv', '2016Blizzard_Summary.csv', '2018Michael_Summary.csv', '2019Dorian_Summary.csv'])

In [8]:
df_all=pd.concat(df_dict.values(), ignore_index=True) 
required_df=df_all[['text','label','filename']]
required_df.head()
# id2label={0:"Negative",1:"Positive"}
# required_df=required_df.replace({"label": id2label})
def clean_text(line):
  clean_text=re.sub(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*','',line)
  clean_text = clean_text.lower()
  clean_text = re.sub("[^a-z0-9]"," ", clean_text)
  clean_text = re.sub("@[A-Za-z0-9]+","",clean_text) #Remove @ sign
  clean_text = " ".join(clean_text.split())
  clean_text = ''.join(c for c in clean_text if c not in emoji.EMOJI_DATA) #Remove Emojis
  return clean_text
required_df['text']=required_df['text'].apply(lambda x: clean_text(x))

<ipython-input-8-78a64492ba5c>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  required_df['text']=required_df['text'].apply(lambda x: clean_text(x))


In [9]:
required_df.head()

,text,label,filename
0,rt uberdogfound please rt food needed for shel...,1,2011Tornado_Summary.csv
1,please rt food needed for shelter in nc in are...,1,2011Tornado_Summary.csv
2,its raining and tornado to be in a coma from e...,1,2011Tornado_Summary.csv
3,lmao damn rt lucecannonn if i don t get any fo...,1,2011Tornado_Summary.csv
4,saved by the tornado warning i m just ready to...,0,2011Tornado_Summary.csv


In [10]:
train, test = train_test_split(required_df, test_size=0.7,stratify=required_df[['filename','label']],random_state=0)

In [17]:
batch_size = 16
model_name_or_path = "microsoft/deberta-base"
task = "mrpc"
peft_type = PeftType.LORA
device = "cuda"
num_epochs = 5


In [ ]:

peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
lr = 3e-4


In [12]:
model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, return_dict=True)
model = get_peft_model(model, peft_config)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
print(model.print_trainable_parameters())

Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.weight', 'classi

trainable params: 296450 || all params: 139488770 || trainable%: 0.21252606930292667
None


In [13]:
class MyClassificationDataset(Dataset):
    #'input_ids', 'attention_mask', 'label'
    def __init__(self, data):
        text, labels = data['text'].values,data['label'].values
        text = [str(i) for i in data['text'].values]
        self.examples = tokenizer(text,truncation=True,padding="max_length",
                                  return_tensors="pt")
        self.labels = torch.tensor(labels, dtype=torch.long)
        

    def __len__(self):
        return len(self.examples["input_ids"])

    def __getitem__(self, index):
        return {'input_ids': self.examples['input_ids'][index],
                'attention_mask':self.examples['attention_mask'][index],
                'labels':self.labels[index]}

train_dataset=MyClassificationDataset(train)
eval_dataset=MyClassificationDataset(test)


def collate_fn(examples):
    return tokenizer.pad(examples, padding="longest", return_tensors="pt")

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=collate_fn, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=collate_fn, batch_size=batch_size, pin_memory=True)

In [14]:
optimizer = AdamW(params=model.parameters(), lr=lr)

# Instantiate scheduler
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0.06 * (len(train_dataloader) * num_epochs),
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [15]:
metric = evaluate.load("accuracy")

In [18]:
model.to(device)
for epoch in range(num_epochs):
    model.train()
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch.to(device)
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to(device)
        with torch.no_grad():
            outputs = model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = metric.compute()
    print(f"epoch {epoch}:", eval_metric)

100%|██████████| 1838/1838 [05:49<00:00,  5.26it/s]


epoch 0: {'accuracy': 0.7511139834688255}


100%|██████████| 1838/1838 [05:49<00:00,  5.27it/s]


epoch 1: {'accuracy': 0.8219327187999592}


100%|██████████| 1838/1838 [05:48<00:00,  5.27it/s]


epoch 2: {'accuracy': 0.8575461750399673}


100%|██████████| 1838/1838 [05:49<00:00,  5.27it/s]


epoch 3: {'accuracy': 0.8610837103302833}


100%|██████████| 1838/1838 [05:48<00:00,  5.27it/s]

epoch 4: {'accuracy': 0.8820368039729243}


In [23]:
#test['predictions']=predictions.detach().cpu()
eval_preds=[]
model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    batch.to(device)
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1)
    predictions, references = predictions, batch["labels"]
    eval_preds.extend(predictions.detach().cpu())
    metric.add_batch(
        predictions=predictions,
        references=references,
    )

100%|██████████| 1838/1838 [05:49<00:00,  5.26it/s]


In [24]:
test['predictions']=eval_preds
test.to_csv("IR_deberta_base_epochs_5.csv")